# Markovian Queues finite state space

In [3]:
import os
import sys
import numpy as np
from copy import deepcopy

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append("../libs")
    
# from libs.stationary_distribution import get_stationary_distribution, check_detailed_balance_condition



def get_stationary_distribution(p: np.ndarray, rates: bool = False) -> np.ndarray:
    n, m = p.shape
    if n != m:
        raise ValueError(f"P is not a square matrix {n}!={m}")

    # Two state markov chain
    if n == 2:
        return np.array([p[1][0], p[0][1]]) / (p[1][0] + p[0][1])
    
    # Continues Time Markov chains with rates
    if rates:
        a = deepcopy(p)
    else:
        a = deepcopy(p - np.eye(n))
    
    a[:, n - 1] = np.ones(n)

    # It will raise LinAlgError if it is Singular matrix or other problems
    return np.linalg.inv(a)[-1]


def is_symmetric(a: np.ndarray) -> bool:
    return np.round(a, 4) == np.round(a.T, 4)


def check_detailed_balance_condition(p: np.ndarray, rates: bool = False) -> bool:
    pi = get_stationary_distribution(p, rates=rates)
    return is_symmetric(pi * p.T)

**4.20. Consider an M/M/s queue with no waiting room. In words, requests for a phone line occur at a rate $\lambda$. If one of the s lines is free, the customer takes it and talks for an exponential amount of time with rate $\mu$. If no lines are free, the customer goes away never to come back. Find the stationary distribution. You do not have to evaluate the normalizing constant.**

$q(n, n+1)=n\mu$, $\pi_k = \frac{c}{k!}\big(\frac{\lambda}{\mu}\big)^k$ where $c^{-1}=\sum_{n=0}^{s}\pi_n$

**4.21. _Two queues in series_. Consider a two station queueing network in which arrivals only occur at the first server and do so at rate 2. If a customer finds server 1 free he enters the system; otherwise he goes away. When a customer is done at the first server he moves on to the second server if it is free and leaves the system if it is not. Suppose that server 1 serves at rate 4 while server 2 serves at rate 2. Formulate a Markov chain model for this system with state space {0, 1, 2, 12} where the state indicates the servers who are busy. In the long run (a) what proportion of customers enter the system? (b) What proportion of the customers visit server 2?**

```
   0    1     2     12
0 -𝜆    𝜆     0     0
1  0  -𝜇1    𝜇1     0
2 𝜇2    0  -(𝜆+𝜇2)  𝜆
12 0   𝜇2    𝜇1  -(𝜇1+𝜇2) 
```

In [2]:
q = np.array([
    [-2, 2, 0, 0],
    [0, -4, 4, 0],
    [2, 0, -4, 2],
    [0, 2, 4, -6],
])
pi = get_stationary_distribution(q, rates=True)
print(f"stationary distribution: {pi}")
print(f"(a) 𝜋(0)+𝜋(2)={pi[0]+pi[2]}")
print(f"(b) 𝜋(0)+𝜋(2)(𝜇2/(𝜇1+𝜇2))=4/9")

stationary distribution: [0.33333333 0.22222222 0.33333333 0.11111111]
(a) 𝜋(0)+𝜋(2)=0.6666666666666666
(b) 𝜋(0)+𝜋(2)(𝜇2/(𝜇1+𝜇2))=4/9


**4.22. Two people who prepare tax forms are working in a store at a local mall. Each has a chair next to his desk where customers can sit and be served. In addition there is one chair where customers can sit and wait. Customers arrive at rate $\lambda$ but will go away if there is already someone sitting in the chair waiting. Suppose that server $i$ requires an exponential amount of time with rate $\mu_i$ and that when both servers are free an arriving customer is equally likely to choose either one. (a) Formulate a Markov chain model for this system with state space {0, 1, 2, 12, 3} where the first four states indicate the servers that are busy while the last indicates that there is a total of three customers in the system: one at each server and one waiting. (b) Consider the special case in which $\lambda = 2, \mu_1 = 3$ and $\mu_2 = 3$. Find the stationary distribution.**

(a) 
```
     0     1     2     12     3
0   -𝜆    𝜆/2   𝜆/2     0     0
1   𝜇1  -(𝜆+𝜇1)  0      𝜆     0
2   𝜇2     0   -(𝜆+𝜇2)  𝜆     0
12   0    𝜇2    𝜇1 -(𝜇1+𝜇2+𝜆) 𝜆
3    0     0     0    𝜇1+𝜇2 -(𝜇1+𝜇2)
```

In [3]:
q = np.array([
    [-2, 1, 1, 0, 0],
    [3, -5, 0, 2, 0],
    [3, 0, -5, 2, 0],
    [0, 3, 3, -8, 2],
    [0, 0, 0, 6, -6]
])
pi = get_stationary_distribution(q, rates=True)
print(f"(b) stationary distribution: {pi}")

(b) stationary distribution: [0.50943396 0.16981132 0.16981132 0.11320755 0.03773585]


**4.23. There are two tennis courts. Pairs of players arrive at rate 3 per hour and play for an exponentially distributed amount of time with mean 1 hour. If there are already two pairs of players waiting new arrivals will leave. Find the stationary distribution for the number of courts occupied.**

Let $X_t$ be the number of pairs waiting or playing for a game, then $\lambda_i=3$ and $\mu_1=1$, and $\mu_2=\mu_3=\mu_4=2$, using DBC as it is _Birth and Death process_.  $\pi_0=c$
$$\pi_1=\frac{3}{1}\pi_0=3c$$
$$\pi_2=\frac{3*3}{2}\pi_1=\frac{9}{2}c$$
$$\pi_3=\frac{3*9}{2*2}\pi_2=\frac{27}{4}c$$
$$\pi_4=\frac{3*27}{2*4}\pi_3=\frac{81}{8}c$$

In [4]:
c = 1/(8+24+36+54+81)
print("Stationary distribution: ", [f"pi_{i}={round(v, 4)}" for i, v in enumerate(c * np.array([8,24,36,54,81]))])

Stationary distribution:  ['pi_0=0.0394', 'pi_1=0.1182', 'pi_2=0.1773', 'pi_3=0.266', 'pi_4=0.399']


**4.24. Carrboro High School is having a car wash to raise money for a spring break trip to Amsterdam. Eight students are working at the car wash. Jill and Kelly are on the sidewalk holding signs advertising the car wash. If there are one, two or three cars getting washed, then the other six students divide themselves up between the cars. If there are four cars, then Jill and Kelly also help, so four cars are being washed by two people each. A car being washed by $m$ students departs at rate $m$ per hour. Cars arrive at rate 12 per hour and enter service with probability $1-k/4$ if $k$ cars are being washed. In short we have the following Markov chain for the number of cars being washed:**
```
   0  1   2   3   4
0 -12 12 
1  6 -15  9
2     6  -12  6
3         6  -9   3  
4             8  -8
```
**(a) Find the stationary distribution. (b) Compute the average number of cars in the system $L$, and the average time a car takes to get washed $W$. (c) Use $\pi(0)$ to compute the expected duration of the busy periods $E_1T_0$. (d) Find $E_1T_0$ by computing $E_xT_0$ for $x=1,2,3$**

In [9]:
q = np.array([
    [-12, 12, 0, 0, 0],
    [6, -15, 9, 0, 0],
    [0, 6, -12, 6, 0],
    [0, 0, 6, -9, 3],
    [0, 0, 0, 8, -8]
])
pi = get_stationary_distribution(q, rates=True)
r = np.array([
    q[1,1:],
    q[2,1:],
    q[3,1:],
    q[4,1:]
])
g = np.dot(-np.linalg.inv(r), np.ones(4))
print(f"(a) stationary distribution: {pi}")
print(f"(b) L={np.round(np.dot(pi, np.arange(5)), 4)}, W={np.round(np.dot(pi, np.arange(5))/12, 4)}")
print(f"(c) E1T0={np.round((1-pi[0])/(12*pi[0]), 4)}")
print(f"(d) ExT0={g}, E1T0={np.round(g[0], 4)}")

(a) stationary distribution: [0.09876543 0.19753086 0.2962963  0.2962963  0.11111111]
(b) L=2.1235, W=0.177
(c) E1T0=0.7604
(d) ExT0=[0.76041667 1.15625    1.38541667 1.51041667], E1T0=0.7604


**2.25. Two teaching assistants work in a homework help room. They can each serve two students, but get less efficient when there are helping two at once. Students arrive at rate 3 but will not go in the room if there are already four students there. To cut a long story short the number of students in the help room is a Markov chain with $Q$ matrix**
```
   0  1  2  3  4
0 -3  3  0  0  0
1  2 -5  3  0  0
2  0  4 -7  3  0
3  0  0  5 -8  3
4  0  0  0  6 -6
```
**(a) Use the detailed balanced condition to find the stationary distribution. (b) Use formula $L=\lambda_aW$ to find the average time W that students spends in the help room. (c) Use $\pi_0$  to compute the expected duration of the busy period $E_1T_0$ by computing $E_xT_0$ for $x=1,2,3$.**

In [11]:
q = np.array([
    [-3, 3, 0, 0, 0],
    [2, -5, 3, 0, 0],
    [0, 4, -7, 3, 0],
    [0, 0, 5, -8, 3],
    [0, 0, 0, 6, -6]
])
pi = get_stationary_distribution(q, rates=True)
r = np.array([
    q[1,1:],
    q[2,1:],
    q[3,1:],
    q[4,1:]
])
g = np.dot(-np.linalg.inv(r), np.ones(4))
print(f"(a) stationary distribution: {pi}")
print(f"(b) L={np.round(np.dot(pi, np.arange(5)), 4)}, W={np.round(np.dot(pi, np.arange(5))/3, 4)}")
print(f"(c) E1T0={np.round((1-pi[0])/(3*pi[0]), 4)}")
print(f" ExT0={g}, E1T0={np.round(g[0], 4)}")

(a) stationary distribution: [0.21563342 0.32345013 0.2425876  0.14555256 0.07277628]
(b) L=1.5364, W=0.5121
(c) E1T0=1.2125
 ExT0=[1.2125     1.6875     1.9875     2.15416667], E1T0=1.2125
